In [56]:
from recipe_scrapers import scrap_me
from bs4 import BeautifulSoup
import urllib
import threading
import time
import csv
import pandas
from multiprocessing.dummy import Pool as ThreadPool 

In [57]:
cuisine = {
'brazilian':'https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=', 
'cajun-and-creole':'https://www.allrecipes.com/recipes/272/us-recipes/cajun-and-creole/?page=',
'southern':'https://www.allrecipes.com/recipes/15876/us-recipes/southern/?page=',
'moroccan':'https://www.allrecipes.com/recipes/1827/world-cuisine/african/north-african/moroccan/?page=',
'filipino':'https://www.allrecipes.com/recipes/696/world-cuisine/asian/filipino/?page=',
'irish':'https://www.allrecipes.com/recipes/706/world-cuisine/european/uk-and-ireland/irish/?page=',
'english':'https://www.allrecipes.com/recipes/705/world-cuisine/european/uk-and-ireland/english/?page=',
'russian':'https://www.allrecipes.com/recipes/716/world-cuisine/european/eastern-european/russian/?page=',
'french':'https://www.allrecipes.com/recipes/721/world-cuisine/european/french/?page=',
'greek':'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=',
'jamaican':'https://www.allrecipes.com/recipes/710/world-cuisine/latin-american/caribbean/jamaican/?page=',
'korean':'https://www.allrecipes.com/recipes/700/world-cuisine/asian/korean/?page=',
'spanish':'https://www.allrecipes.com/recipes/726/world-cuisine/european/spanish/?page=',
'thai':'https://www.allrecipes.com/recipes/702/world-cuisine/asian/thai/?page=',
'vietnamese':'https://www.allrecipes.com/recipes/703/world-cuisine/asian/vietnamese/?page=',
'indian':'https://www.allrecipes.com/recipes/233/world-cuisine/asian/indian/?page=',
'italian':'https://www.allrecipes.com/recipes/723/world-cuisine/european/italian/?page=',
'mexican':'https://www.allrecipes.com/recipes/728/world-cuisine/latin-american/mexican/?=page',
'japanese':'https://www.allrecipes.com/recipes/699/world-cuisine/asian/japanese/?page='
}
# url = 'https://www.allrecipes.com/recipes/703/world-cuisine/asian/vietnamese/?page='

This cell works fine. It speeds up the scraping process by multithreading; however, if you run this it is highly likely that you'll be blocked by the target server.

In [3]:
# urls = []
# for i in range(1,700):
#     urls.append( url+str(i) )
    
# # Make the Pool of workers
# pool = ThreadPool(4) 
# # Open the urls in their own threads
# # and return the results
# results = pool.map(read_url, urls)
# #close the pool and wait for the work to finish 
# pool.close() 
# pool.join() 

Instead we use a slower process; however, a more effective one.

In [5]:
def augment_url(url):
    urls = []
    for i in range(1,2000):
        urls.append( url+str(i) )
    return urls

# augmentedUrls = augment_url(url)

In [6]:
def read_url(url):
    try:
        request = urllib.request
        html_doc = request.urlopen( url ).read()
        print("{}".format(url)+" scrapped!")
        return html_doc
    except:
        print("page does not exist {} end".format(url))
        return "end"
    
# recipePages = []
# for augmentedUrl in augmentedUrls:
#     res = read_url(augmentedUrl)
#     if (res=="end"): break
#     recipePages.append( res )

In [7]:
def readRecipeCards(recipePages):
    links = []
    for page in recipePages:
        soup = BeautifulSoup(page, 'html.parser')
        a_tags = soup.findAll("a", {"class": "fixed-recipe-card__title-link"})
        for a_tag in a_tags:
            links.append(a_tag['href'])
    return links


# recipeUrls = readRecipeCards( recipePages ) 

In [8]:
def extractText(urls,cuisine):
    recipe = []
    res = []

    for url in urls:
        res.append( read_url(url) )
        
    for data in res:
        soup = BeautifulSoup(data, 'html.parser')

        ingredients = soup.findAll("span", {"class": "recipe-ingred_txt added"})
        title = soup.findAll("h1",{"class":"recipe-summary__h1"})

        name = title[0].get_text()
        inge = []
        for ingredient in ingredients:
            inge.append( ingredient.get_text() )

        recipe.append({
            "name":name,
            "cuisine" : cuisine,
            "ingredient":inge
        })
    return recipe
    
# recipes = extractText(recipeUrls,cuisine)

In [9]:
def housekeelping(recipes,cuisine):
    res = pandas.DataFrame(recipes)
    res.to_pickle("allrecipe_"+cuisine)
    res.to_csv('allrecipe_'+cuisine+'.csv')
    
# housekeelping(recipes,"vietnamese")

In [10]:
for key in cuisine:
    augmentedUrls = augment_url(cuisine[key])
    
    recipePages = []
    for augmentedUrl in augmentedUrls:
        res = read_url(augmentedUrl)
        if (res=="end"): break
        recipePages.append( res )
        
    recipeUrls = readRecipeCards( recipePages )
    recipes = extractText(recipeUrls,key)
    housekeelping(recipes,key)


https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=1 scrapped!
https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=2 scrapped!
https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=3 scrapped!
https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=4 scrapped!
https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=5 scrapped!
https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=6 scrapped!
page does not exist https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=7 end
https://www.allrecipes.com/recipe/149479/brazilian-style-moist-coconut-cake/ scrapped!
https://www.allrecipes.com/recipe/58647/brazilian-chicken-with-coconut-milk/ scrapped!
https://www.allrecipes.com/r

https://www.allrecipes.com/recipe/259263/red-grape-and-acai-caipirinha/ scrapped!
https://www.allrecipes.com/recipe/260580/easy-brazilian-pumpkin-coconut-dessert-with-few-ingredients/ scrapped!
https://www.allrecipes.com/recipe/256241/brazilian-christmas-rice/ scrapped!
https://www.allrecipes.com/recipe/254750/brazilian-crab-cakes-casquinha-de-siri/ scrapped!
https://www.allrecipes.com/recipe/254751/brazilian-grilled-salmon-with-mango-salsa/ scrapped!
https://www.allrecipes.com/recipe/255025/espetinho-de-queijo-coalho-com-abacaxi-brazilian-grilled-cheese-with-pineapple/ scrapped!
https://www.allrecipes.com/recipe/254724/caipirinha-de-limo-tradicional-classic-brazilian-cocktail/ scrapped!
https://www.allrecipes.com/recipe/254725/caipirinha-de-morango-com-kiwi-strawberry-kiwi-cocktail/ scrapped!
https://www.allrecipes.com/recipe/221436/brazilian-arroz-de-festa-confetti-rice/ scrapped!
https://www.allrecipes.com/recipe/221438/carrot-farofa/ scrapped!
https://www.allrecipes.com/recipe/2461

https://www.allrecipes.com/recipe/17016/mushroom-and-swiss-chicken/ scrapped!
https://www.allrecipes.com/recipe/25861/anns-shrimp-etouffee/ scrapped!
https://www.allrecipes.com/recipe/151366/costas-french-market-doughnuts-beignets/ scrapped!
https://www.allrecipes.com/recipe/38028/blackened-fish/ scrapped!
https://www.allrecipes.com/recipe/25667/cajun-crab-soup/ scrapped!
https://www.allrecipes.com/recipe/50026/cajun-chicken-lasagna/ scrapped!
https://www.allrecipes.com/recipe/199581/cajun-spiced-pork-chops/ scrapped!
https://www.allrecipes.com/recipe/18377/blackened-tuna/ scrapped!
https://www.allrecipes.com/recipe/48548/cajun-shrimp-orecchiette/ scrapped!
https://www.allrecipes.com/recipe/87079/quick-lemon-dijon-chicken/ scrapped!
https://www.allrecipes.com/recipe/12855/cajun-style-blackened-snapper/ scrapped!
page does not exist https://www.allrecipes.com/recipe/216909/stuffed-jalapeno-firecrackers/ end
https://www.allrecipes.com/recipe/87507/corn-and-crab-bisque/ scrapped!
https://

https://www.allrecipes.com/recipe/22732/crawfish-etouffee-like-maw-maw-used-to-make/ scrapped!
https://www.allrecipes.com/recipe/222581/kale-and-quinoa-with-creole-seasoning/ scrapped!
https://www.allrecipes.com/recipe/149780/cajun-roast-beef/ scrapped!
https://www.allrecipes.com/recipe/29459/shrimp-jambalaya/ scrapped!
https://www.allrecipes.com/recipe/12861/carolina-fish-cakes/ scrapped!
https://www.allrecipes.com/recipe/219644/kes-cajun-dirty-rice/ scrapped!
https://www.allrecipes.com/recipe/233398/summer-squash-and-sausage-stew/ scrapped!
https://www.allrecipes.com/recipe/220863/pressure-cooker-red-beans-and-sausage/ scrapped!
https://www.allrecipes.com/recipe/19936/jalapeno-hamburgers/ scrapped!
https://www.allrecipes.com/recipe/148765/real-new-orleans-style-bbq-shrimp/ scrapped!
https://www.allrecipes.com/recipe/147151/mamas-tomato-gravy/ scrapped!
https://www.allrecipes.com/recipe/213573/crawfish-cornbread/ scrapped!
https://www.allrecipes.com/recipe/44781/creole-mexican-catfish

https://www.allrecipes.com/recipe/13222/jambalaya-i/ scrapped!
https://www.allrecipes.com/recipe/29100/creole-macaroni-and-cheese/ scrapped!
https://www.allrecipes.com/recipe/151105/creole-pan-fried-flat-iron-steak/ scrapped!
https://www.allrecipes.com/recipe/172138/louisiana-crawfish-ya-ya-pasta/ scrapped!
https://www.allrecipes.com/recipe/55359/cajun-chicken-and-rice/ scrapped!
https://www.allrecipes.com/recipe/75923/rattlesnake-pasta/ scrapped!
https://www.allrecipes.com/recipe/86605/cajun-corn-soup/ scrapped!
https://www.allrecipes.com/recipe/216487/new-orleans-eggplant-casserole/ scrapped!
https://www.allrecipes.com/recipe/216886/jambalaya-pot-pie/ scrapped!
https://www.allrecipes.com/recipe/231833/spicy-cajun-cabbage/ scrapped!
https://www.allrecipes.com/recipe/239100/sausage-and-shrimp-gumbo/ scrapped!
https://www.allrecipes.com/recipe/53929/rice-cooker-crawfish-tails/ scrapped!
https://www.allrecipes.com/recipe/20678/ham-and-corn-beignets/ scrapped!
https://www.allrecipes.com/r

https://www.allrecipes.com/recipe/212892/alligator-animal-italian-bread/ scrapped!
https://www.allrecipes.com/recipe/213026/spicy-creamy-cajun-ham-and-black-eyed-peas-salad/ scrapped!
https://www.allrecipes.com/recipe/213555/cajun-style-eggs-benedict/ scrapped!
https://www.allrecipes.com/recipe/103307/shrimp-creole-pronto/ scrapped!
https://www.allrecipes.com/recipe/109881/cajun-crab-rangoon/ scrapped!
https://www.allrecipes.com/recipe/133040/teddys-duck-gumbo/ scrapped!
https://www.allrecipes.com/recipe/212767/shrimp-cheese-ball/ scrapped!
https://www.allrecipes.com/recipe/206680/crawfish-cakes/ scrapped!
https://www.allrecipes.com/recipe/197908/crawfish-boulettes/ scrapped!
https://www.allrecipes.com/recipe/169212/crawfish-macquechou/ scrapped!
https://www.allrecipes.com/recipe/157957/cajun-turkey-stuffing/ scrapped!
https://www.allrecipes.com/recipe/13875/sausage-and-oyster-stuffing/ scrapped!
https://www.allrecipes.com/recipe/22427/crawfish-etouffee-iii/ scrapped!
https://www.allre

https://www.allrecipes.com/recipe/246480/cracklins-deep-fried-pork-belly/ scrapped!


IndexError: list index out of range

In the end all dataframes are concatenated and saved as a file.

In [25]:
frameNames = []
for item in cuisine:
    frameNames.append("allrecipe_"+item)

frames = []
for name in frameNames:
    frames.append(pandas.read_pickle(name))
    
Allrecipie = pandas.concat(frames)
Allrecipie.to_csv('Allrecipe.csv')
Allrecipie.to_pickle("Allrecipe")

In [139]:
Allrecipie.loc[Allrecipie['cuisine'] == 'korean']

,cuisine,ingredient,name
0,korean,"[1/4 cup low-sodium soy sauce, 2 tablespoons w...",Korean Style Salad Dressing
1,korean,[1 pkg. (8 serving size) sweet potato vermicel...,Yummy Korean Glass Noodles (Jap Chae)
2,korean,"[3 pounds beef short ribs, 1/2 cup soy sauce, ...",Kalbi (Korean Marinated Short Ribs)
3,korean,"[2 pounds bone-in chicken thighs, 1/2 cup soy ...",Dak Bulgogi (Korean Barbeque Chicken)
4,korean,"[1 tablespoon vegetable oil, 1/4 cup diced red...",Vegan Korean Kimchi Fried Rice
5,korean,"[1 tablespoon diced ginger root, 1 clove garli...",Korean Take-Out Rice Noodles (Vegan)
6,korean,"[4 racks baby back pork ribs, salt and ground ...",Spicy Korean Ribs
7,korean,"[1 1/2 pounds beef top sirloin, thinly sliced,...",Authentic Korean Bulgogi
8,korean,"[1/4 cup white vinegar, 1/4 teaspoon black pep...",Korean Cucumber Salad
9,korean,"[1 pound flank steak, thinly sliced, 5 tablesp...",Beef Bulgogi


In [95]:
ingredients['ingredient'][0][0]

'1 cup long-grain white rice'

# Preprocessing Data

In [58]:
import gzip
import gensim 
import logging
import pandas
import nltk
import pprint
recipies = pandas.read_pickle("Allrecipe")

In [69]:
for item in recipies["ingredient"]:
    print(item)

['3 cups all-purpose flour', '1 tablespoon baking powder', '3 egg whites', '2 1/2 cups white sugar', '3 egg yolks', '1 cup orange juice', '1 3/4 cups milk', '1 (14 ounce) can coconut milk', '1 (14 ounce) can sweetened condensed milk', '1 cup flaked coconut']
['1 teaspoon ground cumin', '1 teaspoon ground cayenne pepper', '1 teaspoon ground turmeric', '1 teaspoon ground coriander', '4 skinless, boneless chicken breast halves', 'salt and pepper to taste', '2 tablespoons olive oil', '1 onion, chopped', '1 tablespoon minced fresh ginger', '2 jalapeno peppers, seeded and chopped', '2 cloves garlic, minced', '3 tomatoes, seeded and chopped', '1 (14 ounce) can light coconut milk', '1 bunch chopped fresh parsley']
['2 cups cooked white rice', '3/4 cup all-purpose flour', '2 eggs', '1/2 cup chopped fresh parsley', '1/4 cup grated Parmesan cheese', 'salt and ground black pepper to taste', 'oil for frying']
['3 tablespoons brown sugar', '1/2 cup water', '10 bananas, peeled and sliced lengthwise',

['1 (1.5 fluid ounce) jigger vodka', '1 (1.5 fluid ounce) jigger melon liqueur', '1 fluid ounce raspberry flavored liqueur', '4 fluid ounces pineapple juice']
['1 cup semisweet chocolate chips', '1/3 cup butter, melted', '1/2 cup chopped pecans', '2 cups all-purpose flour', '1 teaspoon baking soda', '1 teaspoon salt', '1/2 cup butter', '1 1/2 cups white sugar', '2 eggs', '1 teaspoon vanilla extract', '1 1/4 cups buttermilk', '1 cup heavy whipping cream', '2 tablespoons white sugar', '1 cup flaked coconut']
['1/2 pound cleaned, cooked crawfish tail meat', '1/2 pound jumbo lump crabmeat', '1/2 pound cooked small shrimp, peeled and deveined', '1/2 cup lime juice', '1/2 cup ketchup', '2 tablespoons hot sauce', '2 tablespoons olive oil', '1/3 cup chopped cilantro', '1/2 cup diced red onion (1/4-inch pieces)', '1 cup peeled, seeded, and diced cucumber', '1 cup diced jicama', '1 jalapeno chile pepper, seeded and minced', 'salt to taste', '1 large avocado, diced']
['1 scoop vanilla ice cream',

['1/2 cup semi-sweet chocolate chips', '2 cups heavy cream', '1/4 cup white sugar', '1 pinch salt', '2 teaspoons vanilla extract', '5 egg yolks', '4 tablespoons white sugar']
['1/4 cup all-purpose flour', '1 teaspoon salt', '1/2 teaspoon ground black pepper', '2 pounds cubed stew meat', '4 tablespoons butter', '1 onion, chopped', '2 carrots, chopped', '1 clove garlic, minced', '2 cups red wine', '1 bay leaf', '3 tablespoons chopped fresh parsley', '1/2 teaspoon dried thyme', '1 (6 ounce) can sliced mushrooms', '1 (16 ounce) can canned onions']
['2 cups heavy cream', '3 tablespoons cultured buttermilk']
['4 onions, chopped', '3 tablespoons butter', '3 tablespoons all-purpose flour', '1 teaspoon ground black pepper', '1 teaspoon white sugar', '3 (10.5 ounce) cans beef broth', '1 1/4 cups water', '1/2 teaspoon dried parsley', '1/4 teaspoon dried thyme', '1 cup white wine', '1 French baguette, cut into 1/2 inch slices', '8 ounces shredded mozzarella cheese']
['6 bone-in, skin-on chicken th

['Marinade:', '1 pound skinless, boneless chicken thighs, quartered', '1/2 yellow onion, grated', '4 cloves garlic, minced', '1 teaspoon fine salt', '1/2 teaspoon freshly ground black pepper', 'Batter:', '3/4 cup cornstarch', '1/2 cup self-rising flour', '1 teaspoon white sugar', '1/2 teaspoon ground black pepper', '1/4 teaspoon salt', '1 cup very cold water, or as needed', 'oil, or as needed']
['2 heads napa cabbage', '1 1/4 cups sea salt', '1 tablespoon fish sauce', '5 green onions, chopped', '1/2 small white onion, minced', '2 cloves garlic, pressed', '2 tablespoons white sugar', '1 teaspoon ground ginger', '5 tablespoons Korean chile powder']
['1/4 yellow onion, thinly sliced', '2 green onions, chopped, dark green parts separated from white and light green parts', '1/3 cup soy sauce', '3 tablespoons white sugar', '2 tablespoons toasted sesame seeds', '3 cloves garlic, minced', '1 tablespoon sesame oil', '1/4 teaspoon Korean red pepper flakes', '1/4 teaspoon minced fresh ginger', '1

['3 tablespoons mustard seed', '5 green chile peppers, diced', '1/4 cup vegetable oil', '4 medium onions, chopped', '1 teaspoon chili powder', '1 teaspoon ground turmeric', 'salt to taste', '1/3 cup water', '2 pounds salmon, cut into chunks']
['2 large dried red chile peppers (optional)', '2 bunches fresh spinach, washed and chopped', '1 bunch mustard greens, washed and chopped', '2 tablespoons ghee (clarified butter)', '1/2 teaspoon cumin seeds', '1/4 cup finely chopped onion', '1/2 teaspoon ginger paste', '2 teaspoons garlic paste', '1/2 tomato, chopped', 'salt to taste', '1/2 teaspoon white sugar, or to taste', '1/4 cup water (optional)']
['2 tablespoons vegetable oil', '1 teaspoon ground cumin', '1/2 teaspoon ground dried red chilies', '2 serrano chile peppers, seeded and chopped', '2 large onions, chopped', '2 (15 ounce) cans garbanzo beans (chickpeas), drained', '3 tablespoons lemon juice', '2/3 cup water', 'salt to taste', '3 tablespoons minced fresh ginger root', '2 large tomat

['1/2 cup chopped onion', '1/2 cup chopped green bell pepper', '1/2 cup red bell pepper, chopped', '1 1/2 cups fresh sliced mushrooms', '4 tablespoons butter', '1 (10.75 ounce) can condensed cream of chicken soup', '1/3 cup milk']
['1 pound Italian sausage roll', '8 lasagna noodles, broken into pieces', '1 (28 ounce) can crushed tomatoes', '1 (8 ounce) can tomato sauce', '2 cups torn fresh spinach', '1/2 cup ricotta cheese', '4 ounces fresh mozzarella cheese, sliced', '1/4 cup grated Parmesan cheese']
['1 pound skinless, boneless chicken breast - cooked and diced', '4 cups spaghetti sauce', '2 tablespoons hot sauce', '2 tablespoons apple cider vinegar', '1 1/2 cups water', '1 teaspoon garlic powder', '1 small onion, chopped', '1 small green bell pepper, chopped', '1 (6 ounce) can mushrooms, drained', '1 egg, beaten', '1 (15 ounce) container ricotta cheese', '12 uncooked lasagna noodles', '2 cups shredded mozzarella cheese', '3/4 cup crumbled blue cheese']
['12 roma (plum) tomatoes, cho

['2 cups halved grape tomatoes', '1 tablespoon olive oil', '1 tablespoon sweetened rice vinegar', '1/4 teaspoon garlic powder', '1/4 teaspoon dried oregano, or to taste', 'salt to taste (optional)']
['2 skinless, boneless chicken breast halves', 'salt and ground black pepper to taste', '6 tablespoons olive oil, divided', '4 zucchini, cut into 1 1/2-inch sticks', '1 (15 ounce) can garbanzo beans, drained and rinsed', '1 (6 ounce) can black olives, drained and sliced', '1 (14 ounce) can artichoke hearts, drained and chopped', '1/2 cup grated Parmesan cheese']
['1 butternut squash, halved and seeded', '1 tablespoon olive oil, divided, or as needed', '1 (8 ounce) package sliced fresh mushrooms', '1 teaspoon minced garlic', '1 onion, chopped', '1 (10 ounce) package frozen chopped spinach, thawed and drained', '7 cups milk, divided', '1/2 cup cornstarch', '2 cups grated Parmesan cheese, divided', '1 tablespoon honey', '1/4 teaspoon ground nutmeg', '4 fresh sage leaves, shredded, or to taste'

['Bechamel Sauce:', '1/4 cup butter', '1/2 cup all-purpose flour', '3 1/3 cups milk', '1 pinch ground nutmeg', '1 pinch salt', 'Lasagna:', '2 tablespoons extra-virgin olive oil', '1 clove garlic, crushed', '1 pound fresh artichoke hearts, thinly sliced', '1/2 cup white wine', '1 teaspoon dried dill', 'salt and ground black pepper to taste', '7 ounces pancetta (Italian bacon), cubed', '12 lasagna noodles', '1 cup grated Parmesan cheese']
['3/4 cup vegetable oil', '4 yellow onions, cut into matchstick-size pieces', '1 1/2 cups minced garlic', '1 cup butter', '2 1/2 cups warm water', '1/2 cup olive oil', '6 1/2 cups bread flour', '1/2 cup white sugar', '1 1/2 tablespoons active dry yeast', '1 tablespoon coarse kosher salt', '3/4 cup minced fresh parsley', '1 1/2 tablespoons minced fresh rosemary', '1/2 teaspoon dried marjoram', '2 tablespoons all-purpose flour, or as needed (optional)', '1 1/2 cups crumbled blue cheese', '1 tablespoon vegetable oil, or as needed', '2 tablespoons water, or

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [65]:
s = []
recipies["ingredient"]
def func(row):
    for item in row['ingredient']:
        s.append(item.split())
    
recipies.apply(func,axis=1)
#     s.append(i.)
#     pprint.pprint(i)

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
       ... 
216    None
217    None
218    None
219    None
220    None
221    None
222    None
223    None
224    None
225    None
226    None
227    None
228    None
229    None
230    None
231    None
232    None
233    None
234    None
235    None
236    None
237    None
238    None
239    None
240    None
241    None
242    None
243    None
244    None
245    None
Length: 10063, dtype: object

In [68]:
s[0][0]

'3'

In [3]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True) 
# sentences = open("101RecipesForTheDeepFryer.txt","r")
# gensim.utils.simple_preprocess(sentences)
# sentences.read()
# model = Word2Vec(sentences, size=100, window=50, min_count=1, workers=4)


2018-05-02 13:23:10,296 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin
2018-05-02 13:23:53,290 : INFO : loaded (3000000, 300) matrix from GoogleNews-vectors-negative300.bin


In [141]:
# model.wv['eggs']
words = list(word_vectors.wv.vocab)
# pprint.pprint(words)
w1 = ['pork']
# word_vectors.wv['tomato']
word_vectors.wv.most_similar(positive=w1,topn=6)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


[('meat', 0.718697726726532),
 ('Pork', 0.6871966123580933),
 ('beef', 0.6601365208625793),
 ('chicken', 0.6541997790336609),
 ('meats', 0.6151372790336609),
 ('lamb', 0.6107682585716248)]